<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/adverserial_attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/insertion_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/removal_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/adverserial_attacks_functions.py'
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 5.68MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 3.34MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 3.64MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 14.4MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 9.40MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *


torch.manual_seed(0)



cuda
cuda


In [4]:
# Open the .pkl file
with open('/content/insertion_array.pkl', 'rb') as f:
    # Load the object
    insertion_array = pickle.load(f)

# Close the file
f.close()

insertion_array = torch.tensor(insertion_array).to(device)
print(len(insertion_array))

# Open the .pkl file
with open('/content/removal_array.pkl', 'rb') as f:
    # Load the object
    removal_array = pickle.load(f)

# Close the file
f.close()

removal_array = torch.tensor(removal_array).to(device)
print(len(removal_array))


10000
10000


In [5]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [6]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [7]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [8]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [9]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [10]:
model_evaluation(model_DNN, test_loader, device)

Test Accuracy: 0.9912
Test balanced Accuracy: 0.9785
Precision: 0.9637
Recall: 0.9619
F1-score: 0.9628
True Positives (TP): 1087
True Negatives (TN): 8391
False Positives (FP): 41
False Negatives (FN): 43
False Negative Rate (FNR): 3.8053
False Positive Rate (FPR): 0.4862


In [11]:
model_evaluation(model_AT_rFGSM,test_loader,device)

Test Accuracy: 0.9893
Test balanced Accuracy: 0.9710
Precision: 0.9622
Recall: 0.9469
F1-score: 0.9545
True Positives (TP): 1070
True Negatives (TN): 8390
False Positives (FP): 42
False Negatives (FN): 60
False Negative Rate (FNR): 5.3097
False Positive Rate (FPR): 0.4981


In [12]:
model_evaluation(model_AT_rFGSM_weightedLoss,test_loader,device)

Test Accuracy: 0.9872
Test balanced Accuracy: 0.9782
Precision: 0.9286
Recall: 0.9664
F1-score: 0.9471
True Positives (TP): 1092
True Negatives (TN): 8348
False Positives (FP): 84
False Negatives (FN): 38
False Negative Rate (FNR): 3.3628
False Positive Rate (FPR): 0.9962


In [ ]:
# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),
    (mimicry, {'removal_array':removal_array, "trials":1, 'is_report_loss_diff':False}),
    (mimicry, {'removal_array':removal_array, "trials":10, 'is_report_loss_diff':False}),
    (mimicry, {'removal_array':removal_array, "trials":30, 'is_report_loss_diff':False}),
    (pgd, {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (gkde,{'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (gkde,{'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (gkde,{'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (PGD_Max2, {'insertion_array':insertion_array, 'removal_array':removal_array,'steps_max':1,'is_sample':False, 'varepsilon':1e-20}),
    (PGD_Max2, {'insertion_array':insertion_array, 'removal_array':removal_array,'steps_max':5,'is_sample':False, 'varepsilon':1e-20}),
    (StepwiseMax_onestep2, {'insertion_array':insertion_array, 'removal_array':removal_array,'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_DNN, attack_func, device, **attack_params)
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    adv_predict(test_loader, model_AT_rFGSM_weightedLoss, attack_func, device, **attack_params)
    print('********************************************')  # Print an empty line for separation

Running attack: dfgsm_k with parameters: {'k': 100, 'epsilon': 0.02, 'random': True, 'is_report_loss_diff': False}
Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.98%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.84%.
********************************************
Running attack: bga_k with parameters: {'k': 100, 'is_report_loss_diff': False}
Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.81%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 94.16%.
********************************************
Running attack: bca_k with parameters: {'k': 100, 'is_report_loss_diff': False}
Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 90.09%.
Accuracy of just malwares (without attack

In [15]:
# Mimicry×30
attack_params = {'removal_array':removal_array, "trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0885%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 41.77%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 46.64%.


In [39]:
# Mimicry×30
attack_params = {'removal_array':removal_array, "trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 2.035%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 48.67%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 47.43%.


In [26]:
attack_params =  {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

PGD l2: Attack effectiveness 51.852%.
PGD l2: Attack effectiveness 60.656%.
PGD l2: Attack effectiveness 68.000%.
PGD l2: Attack effectiveness 61.194%.
PGD l2: Attack effectiveness 56.338%.
PGD l2: Attack effectiveness 55.556%.
PGD l2: Attack effectiveness 61.702%.
PGD l2: Attack effectiveness 51.786%.
PGD l2: Attack effectiveness 54.545%.
PGD l2: Attack effectiveness 55.172%.
PGD l2: Attack effectiveness 63.333%.
PGD l2: Attack effectiveness 56.863%.
PGD l2: Attack effectiveness 52.857%.
PGD l2: Attack effectiveness 47.692%.
PGD l2: Attack effectiveness 57.812%.
PGD l2: Attack effectiveness 53.425%.
PGD l2: Attack effectiveness 57.971%.
PGD l2: Attack effectiveness 50.000%.
PGD l2: Attack effectiveness 46.939%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 42.21%.


In [27]:

attack_params = {'bandwidth': 1.,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 61.111%.
PGD l2: Attack effectiveness 47.541%.
PGD l2: Attack effectiveness 44.000%.
PGD l2: Attack effectiveness 40.299%.
PGD l2: Attack effectiveness 47.887%.
PGD l2: Attack effectiveness 46.296%.
PGD l2: Attack effectiveness 57.447%.
PGD l2: Attack effectiveness 55.357%.
PGD l2: Attack effectiveness 47.273%.
PGD l2: Attack effectiveness 58.621%.
PGD l2: Attack effectiveness 56.667%.
PGD l2: Attack effectiveness 49.020%.
PGD l2: Attack effectiveness 50.000%.
PGD l2: Attack effectiveness 63.077%.
PGD l2: Attack effectiveness 43.750%.
PGD l2: Attack effectiveness 52.055%.
PGD l2: Attack effectiveness 57.971%.
PGD l2: Attack effectiveness 57.143%.
PGD l2: Attack effectiveness 53.061%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 47.35%.


In [67]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 70.370%.
PGD l2: Attack effectiveness 73.770%.
PGD l2: Attack effectiveness 68.000%.
PGD l2: Attack effectiveness 74.627%.
PGD l2: Attack effectiveness 53.521%.
PGD l2: Attack effectiveness 57.407%.
PGD l2: Attack effectiveness 80.851%.
PGD l2: Attack effectiveness 55.357%.
PGD l2: Attack effectiveness 67.273%.
PGD l2: Attack effectiveness 72.414%.
PGD l2: Attack effectiveness 75.000%.
PGD l2: Attack effectiveness 64.706%.
PGD l2: Attack effectiveness 61.429%.
PGD l2: Attack effectiveness 73.846%.
PGD l2: Attack effectiveness 60.938%.
PGD l2: Attack effectiveness 73.973%.
PGD l2: Attack effectiveness 65.217%.
PGD l2: Attack effectiveness 62.500%.
PGD l2: Attack effectiveness 57.143%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 31.42%.


In [26]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 75.926%.
PGD l2: Attack effectiveness 81.967%.
PGD l2: Attack effectiveness 74.000%.
PGD l2: Attack effectiveness 80.597%.
PGD l2: Attack effectiveness 76.056%.
PGD l2: Attack effectiveness 75.926%.
PGD l2: Attack effectiveness 68.085%.
PGD l2: Attack effectiveness 76.786%.
PGD l2: Attack effectiveness 63.636%.
PGD l2: Attack effectiveness 74.138%.
PGD l2: Attack effectiveness 73.333%.
PGD l2: Attack effectiveness 72.549%.
PGD l2: Attack effectiveness 65.714%.
PGD l2: Attack effectiveness 72.308%.
PGD l2: Attack effectiveness 70.312%.
PGD l2: Attack effectiveness 76.712%.
PGD l2: Attack effectiveness 76.812%.
PGD l2: Attack effectiveness 87.500%.
PGD l2: Attack effectiveness 75.510%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 23.45%.


In [30]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 70.370%.
PGD l2: Attack effectiveness 72.131%.
PGD l2: Attack effectiveness 68.000%.
PGD l2: Attack effectiveness 74.627%.
PGD l2: Attack effectiveness 53.521%.
PGD l2: Attack effectiveness 57.407%.
PGD l2: Attack effectiveness 80.851%.
PGD l2: Attack effectiveness 55.357%.
PGD l2: Attack effectiveness 67.273%.
PGD l2: Attack effectiveness 72.414%.
PGD l2: Attack effectiveness 75.000%.
PGD l2: Attack effectiveness 64.706%.
PGD l2: Attack effectiveness 61.429%.
PGD l2: Attack effectiveness 72.308%.
PGD l2: Attack effectiveness 57.812%.
PGD l2: Attack effectiveness 73.973%.
PGD l2: Attack effectiveness 65.217%.
PGD l2: Attack effectiveness 62.500%.
PGD l2: Attack effectiveness 57.143%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 31.77%.


In [45]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 66.667%.
PGD l2: Attack effectiveness 70.492%.
PGD l2: Attack effectiveness 68.000%.
PGD l2: Attack effectiveness 64.179%.
PGD l2: Attack effectiveness 67.606%.
PGD l2: Attack effectiveness 61.111%.
PGD l2: Attack effectiveness 68.085%.
PGD l2: Attack effectiveness 57.143%.
PGD l2: Attack effectiveness 70.909%.
PGD l2: Attack effectiveness 58.621%.
PGD l2: Attack effectiveness 70.000%.
PGD l2: Attack effectiveness 50.980%.
PGD l2: Attack effectiveness 55.714%.
PGD l2: Attack effectiveness 69.231%.
PGD l2: Attack effectiveness 48.438%.
PGD l2: Attack effectiveness 64.384%.
PGD l2: Attack effectiveness 59.420%.
PGD l2: Attack effectiveness 64.286%.
PGD l2: Attack effectiveness 61.224%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 35.4%.


In [32]:
attack_params = {'bandwidth': 0.65,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 70.370%.
PGD l2: Attack effectiveness 67.213%.
PGD l2: Attack effectiveness 70.000%.
PGD l2: Attack effectiveness 71.642%.
PGD l2: Attack effectiveness 63.380%.
PGD l2: Attack effectiveness 68.519%.
PGD l2: Attack effectiveness 68.085%.
PGD l2: Attack effectiveness 66.071%.
PGD l2: Attack effectiveness 74.545%.
PGD l2: Attack effectiveness 65.517%.
PGD l2: Attack effectiveness 81.667%.
PGD l2: Attack effectiveness 66.667%.
PGD l2: Attack effectiveness 70.000%.
PGD l2: Attack effectiveness 70.769%.
PGD l2: Attack effectiveness 57.812%.
PGD l2: Attack effectiveness 73.973%.
PGD l2: Attack effectiveness 66.667%.
PGD l2: Attack effectiveness 64.286%.
PGD l2: Attack effectiveness 55.102%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 32.48%.


In [33]:
attack_params = {'bandwidth': 0.55,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 66.667%.
PGD l2: Attack effectiveness 63.934%.
PGD l2: Attack effectiveness 68.000%.
PGD l2: Attack effectiveness 71.642%.
PGD l2: Attack effectiveness 63.380%.
PGD l2: Attack effectiveness 55.556%.
PGD l2: Attack effectiveness 74.468%.
PGD l2: Attack effectiveness 62.500%.
PGD l2: Attack effectiveness 65.455%.
PGD l2: Attack effectiveness 67.241%.
PGD l2: Attack effectiveness 73.333%.
PGD l2: Attack effectiveness 60.784%.
PGD l2: Attack effectiveness 64.286%.
PGD l2: Attack effectiveness 63.077%.
PGD l2: Attack effectiveness 57.812%.
PGD l2: Attack effectiveness 67.123%.
PGD l2: Attack effectiveness 57.971%.
PGD l2: Attack effectiveness 66.071%.
PGD l2: Attack effectiveness 57.143%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 33.63%.


In [34]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 500,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 70.370%.
PGD l2: Attack effectiveness 73.770%.
PGD l2: Attack effectiveness 62.000%.
PGD l2: Attack effectiveness 70.149%.
PGD l2: Attack effectiveness 59.155%.
PGD l2: Attack effectiveness 59.259%.
PGD l2: Attack effectiveness 78.723%.
PGD l2: Attack effectiveness 58.929%.
PGD l2: Attack effectiveness 65.455%.
PGD l2: Attack effectiveness 72.414%.
PGD l2: Attack effectiveness 75.000%.
PGD l2: Attack effectiveness 60.784%.
PGD l2: Attack effectiveness 65.714%.
PGD l2: Attack effectiveness 70.769%.
PGD l2: Attack effectiveness 60.938%.
PGD l2: Attack effectiveness 69.863%.
PGD l2: Attack effectiveness 65.217%.
PGD l2: Attack effectiveness 64.286%.
PGD l2: Attack effectiveness 59.184%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 32.04%.


In [35]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1500,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l2: Attack effectiveness 66.667%.
PGD l2: Attack effectiveness 72.131%.
PGD l2: Attack effectiveness 64.000%.
PGD l2: Attack effectiveness 68.657%.
PGD l2: Attack effectiveness 54.930%.
PGD l2: Attack effectiveness 61.111%.
PGD l2: Attack effectiveness 74.468%.
PGD l2: Attack effectiveness 58.929%.
PGD l2: Attack effectiveness 65.455%.
PGD l2: Attack effectiveness 72.414%.
PGD l2: Attack effectiveness 80.000%.
PGD l2: Attack effectiveness 60.784%.
PGD l2: Attack effectiveness 65.714%.
PGD l2: Attack effectiveness 70.769%.
PGD l2: Attack effectiveness 64.062%.
PGD l2: Attack effectiveness 75.342%.
PGD l2: Attack effectiveness 63.768%.
PGD l2: Attack effectiveness 66.071%.
PGD l2: Attack effectiveness 53.061%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 34.34%.


In [36]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM, gkde, device, **attack_params)

PGD l2: Attack effectiveness 70.370%.
PGD l2: Attack effectiveness 63.934%.
PGD l2: Attack effectiveness 56.000%.
PGD l2: Attack effectiveness 65.672%.
PGD l2: Attack effectiveness 67.606%.
PGD l2: Attack effectiveness 61.111%.
PGD l2: Attack effectiveness 74.468%.
PGD l2: Attack effectiveness 53.571%.
PGD l2: Attack effectiveness 65.455%.
PGD l2: Attack effectiveness 65.517%.
PGD l2: Attack effectiveness 81.667%.
PGD l2: Attack effectiveness 66.667%.
PGD l2: Attack effectiveness 71.429%.
PGD l2: Attack effectiveness 72.308%.
PGD l2: Attack effectiveness 54.688%.
PGD l2: Attack effectiveness 72.603%.
PGD l2: Attack effectiveness 56.522%.
PGD l2: Attack effectiveness 64.286%.
PGD l2: Attack effectiveness 67.347%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 33.98%.


In [37]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD l1: Attack effectiveness 59.259%.
PGD l1: Attack effectiveness 62.295%.
PGD l1: Attack effectiveness 62.000%.
PGD l1: Attack effectiveness 55.224%.
PGD l1: Attack effectiveness 57.746%.
PGD l1: Attack effectiveness 55.556%.
PGD l1: Attack effectiveness 57.447%.
PGD l1: Attack effectiveness 58.929%.
PGD l1: Attack effectiveness 60.000%.
PGD l1: Attack effectiveness 56.897%.
PGD l1: Attack effectiveness 68.333%.
PGD l1: Attack effectiveness 45.098%.
PGD l1: Attack effectiveness 58.571%.
PGD l1: Attack effectiveness 73.846%.
PGD l1: Attack effectiveness 50.000%.
PGD l1: Attack effectiveness 63.014%.
PGD l1: Attack effectiveness 59.420%.
PGD l1: Attack effectiveness 60.714%.
PGD l1: Attack effectiveness 65.306%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 40.44%.


In [38]:
attack_params = {'bandwidth': 0.6,'penalty_factor': 1000,'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, gkde, device, **attack_params)

PGD linf: Attack effectiveness 40.741%.
PGD linf: Attack effectiveness 44.262%.
PGD linf: Attack effectiveness 32.000%.
PGD linf: Attack effectiveness 29.851%.
PGD linf: Attack effectiveness 35.211%.
PGD linf: Attack effectiveness 29.630%.
PGD linf: Attack effectiveness 42.553%.
PGD linf: Attack effectiveness 35.714%.
PGD linf: Attack effectiveness 41.818%.
PGD linf: Attack effectiveness 39.655%.
PGD linf: Attack effectiveness 43.333%.
PGD linf: Attack effectiveness 31.373%.
PGD linf: Attack effectiveness 38.571%.
PGD linf: Attack effectiveness 56.923%.
PGD linf: Attack effectiveness 28.125%.
PGD linf: Attack effectiveness 43.836%.
PGD linf: Attack effectiveness 34.783%.
PGD linf: Attack effectiveness 41.071%.
PGD linf: Attack effectiveness 42.857%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 60.35%.


In [39]:
attack_params =  {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : True}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

PGD linf: Attack effectiveness 5.556%.
PGD linf: Attack effectiveness 8.197%.
PGD linf: Attack effectiveness 8.000%.
PGD linf: Attack effectiveness 5.970%.
PGD linf: Attack effectiveness 11.268%.
PGD linf: Attack effectiveness 9.259%.
PGD linf: Attack effectiveness 14.894%.
PGD linf: Attack effectiveness 7.143%.
PGD linf: Attack effectiveness 5.455%.
PGD linf: Attack effectiveness 8.621%.
PGD linf: Attack effectiveness 16.667%.
PGD linf: Attack effectiveness 13.725%.
PGD linf: Attack effectiveness 10.000%.
PGD linf: Attack effectiveness 4.615%.
PGD linf: Attack effectiveness 4.688%.
PGD linf: Attack effectiveness 10.959%.
PGD linf: Attack effectiveness 2.899%.
PGD linf: Attack effectiveness 3.571%.
PGD linf: Attack effectiveness 8.163%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 90.44%.


In [ ]:

# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),
    (mimicry, {'removal_array':removal_array, "trials":30, 'is_report_loss_diff':False}),
    (pgd, {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.5, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'insertion_array':insertion_array, 'removal_array':removal_array,'k': 50, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (PGD_Max2, {'insertion_array':insertion_array, 'removal_array':removal_array,'steps_max':1,'is_sample':False, 'varepsilon':1e-20}),
    (PGD_Max2, {'insertion_array':insertion_array, 'removal_array':removal_array,'steps_max':5,'is_sample':False, 'varepsilon':1e-20}),
    (StepwiseMax_onestep2, {'insertion_array':insertion_array, 'removal_array':removal_array,'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_DNN, attack_func, device, **attack_params)
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    adv_predict(test_loader, model_AT_rFGSM_weightedLoss, attack_func, device, **attack_params)
    print('********************************************')  # Print an empty line for separation

Running attack: dfgsm_k with parameters: {'k': 100, 'epsilon': 0.02, 'random': True, 'is_report_loss_diff': False}
Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.63%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.84%.
********************************************
Running attack: bga_k with parameters: {'k': 100, 'is_report_loss_diff': False}


KeyboardInterrupt: 

In [ ]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

In [68]:
for x,y in test_loader:
  print(x.shape)
  break

torch.Size([512, 10000])


In [69]:
bens = x[y.squeeze()==0]
mals = x[y.squeeze()==1]
print(bens.shape)
print(mals.shape)

torch.Size([458, 10000])
torch.Size([54, 10000])


In [ ]:
model_AT_rFGSM(bens.to(torch.float32).to(device))

In [ ]:
malware_sample = mals[0:1,:]
malware_sample.shape

torch.Size([1, 10000])

In [ ]:
benign_samples = []
for x_batch, y_batch in test_loader:
  benign_samples.append(x_batch[y_batch.squeeze() == 0])

ben_x = torch.cat(benign_samples, dim=0).to(device)

print(ben_x.shape)
del benign_samples

torch.Size([8432, 10000])


In [ ]:
# Suppose you want to extract a subset of size subset_size from ben_x
subset_size = 500  # Adjust this value according to your requirement

# Get the total number of samples in ben_x
total_samples = ben_x.size(0)

# Generate random indices for the subset
subset_indices = np.random.choice(total_samples, subset_size, replace=False)

# Select the subset of data using the random indices
subset_data = ben_x[subset_indices]

print(subset_data.shape)

torch.Size([500, 10000])


In [ ]:
import torch

# Define the multivariate Gaussian kernel function
def gaussian_kernel(u):
    """
    Compute the value of the Gaussian kernel function at point u.

    Parameters:
        u (torch.Tensor): Standardized distance between data points and evaluation points.

    Returns:
        torch.Tensor: Array of Gaussian kernel values.
    """
    d = u.shape[1]  # Number of dimensions (features) of the data
    # normalization constant for the Gaussian kernel in d dimensions
    #normalization = (2 * np.pi) ** (-d / 2)
    normalization = 10.  # normalization constant for the Gaussian kernel in d dimensions
    return normalization * torch.exp(-0.5 * torch.sum(u**2, dim=-1))

# Multidimensional KDE implementation
def KDE(x, data, bandwidth):
    """
    Compute the kernel density estimate (KDE) for given data points.

    Parameters:
        x (torch.Tensor): Points at which to evaluate the KDE (shape: [num_samples, num_dimensions]).
        data (torch.Tensor): Data points used to estimate the density (shape: [num_data_points, num_dimensions]).
        bandwidth (float): Bandwidth parameter for the KDE.

    Returns:
        torch.Tensor: Density estimate at each point in x (shape: [num_samples,]).
    """
    n = data.shape[0]  # Number of data points
    d = x.shape[1]  # Dimensionality of the data

    # Convert bandwidth to tensor
    bandwidth_tensor = torch.tensor(bandwidth)

    # Calculate standardized distances for all data points
    u = (x.unsqueeze(1) - data) / bandwidth


    # Compute Gaussian kernel contributions for all data points
    normalization = 10.
    kernel_contributions = normalization * torch.exp(-0.5 * torch.sum(u**2, dim=-1))

    # Sum contributions across all data points
    estimate = torch.mean(kernel_contributions, dim=1)

    # Normalize the density estimate by the number of points and the bandwidth raised to the dimensionality
    #estimate /= (bandwidth_tensor ** d)

    return estimate


In [ ]:
pgd(mals.to(torch.float32), y[y.squeeze()==1], model_AT_rFGSM_weightedLoss, insertion_array, removal_array, k=50, step_length=0.5, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)

PGD linf: Attack effectiveness 0.000%.


tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.]])

In [ ]:
gkde(mals.to(torch.float32), y[y.squeeze()==1], model_AT_rFGSM_weightedLoss, subset_data, 1., 1000, insertion_array, removal_array, k=50, step_length=0.05, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)

3
4
5
3
4
5
3


KeyboardInterrupt: 

In [ ]:
gkde(mals,mals_label,model_AT_rFGSM,bens,1.,1000.,insertion_array, removal_array, k=200, step_length=0.05, norm='l2', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)

In [ ]:
import torch

# Define the multivariate Gaussian kernel function
def gaussian_kernel(u):
    """
    Compute the value of the Gaussian kernel function at point u.

    Parameters:
        u (torch.Tensor): Standardized distance between data points and evaluation points.

    Returns:
        torch.Tensor: Array of Gaussian kernel values.
    """
    d = u.shape[1]  # Number of dimensions (features) of the data
    # normalization constant for the Gaussian kernel in d dimensions
    #normalization = (2 * np.pi) ** (-d / 2)
    normalization = 10.  # normalization constant for the Gaussian kernel in d dimensions
    return normalization * torch.exp(-0.5 * torch.sum(u**2, dim=-1))

# Multidimensional KDE implementation
def KDE(x, data, bandwidth):
    """
    Compute the kernel density estimate (KDE) for given data points.

    Parameters:
        x (torch.Tensor): Points at which to evaluate the KDE (shape: [num_samples, num_dimensions]).
        data (torch.Tensor): Data points used to estimate the density (shape: [num_data_points, num_dimensions]).
        bandwidth (float): Bandwidth parameter for the KDE.

    Returns:
        torch.Tensor: Density estimate at each point in x (shape: [num_samples,]).
    """
    n = data.shape[0]  # Number of data points
    d = x.shape[1]  # Dimensionality of the data

    # Convert bandwidth to tensor
    bandwidth_tensor = torch.tensor(bandwidth)

    estimate = torch.zeros(x.shape[0],dtype=torch.float)
    for sample in data:
        # Calculate standardized distances for all data points
        u = (x[:, None, :] - sample) / bandwidth_tensor

        # Compute Gaussian kernel contributions for all data points
        kernel_contributions = gaussian_kernel(u)

        # Sum contributions across all data points
        estimate = torch.mean(kernel_contributions, dim=1)

        # Normalize the density estimate by the number of points and the bandwidth raised to the dimensionality
        #estimate /= (bandwidth_tensor ** d)

    return estimate


In [ ]:
import torch

# Define the multivariate Gaussian kernel function
def gaussian_kernel(u):
    """
    Compute the value of the Gaussian kernel function at point u.

    Parameters:
        u (torch.Tensor): Standardized distance between data points and evaluation points.

    Returns:
        torch.Tensor: Array of Gaussian kernel values.
    """
    d = u.shape[1]  # Number of dimensions (features) of the data
    # normalization constant for the Gaussian kernel in d dimensions
    #normalization = (2 * np.pi) ** (-d / 2)
    normalization = 10.  # normalization constant for the Gaussian kernel in d dimensions
    return normalization * torch.exp(-0.5 * torch.sum(u**2, dim=-1))

# Multidimensional KDE implementation
def KDE(x, data, bandwidth):
    """
    Compute the kernel density estimate (KDE) for given data points.

    Parameters:
        x (torch.Tensor): Points at which to evaluate the KDE (shape: [num_samples, num_dimensions]).
        data (torch.Tensor): Data points used to estimate the density (shape: [num_data_points, num_dimensions]).
        bandwidth (float): Bandwidth parameter for the KDE.

    Returns:
        torch.Tensor: Density estimate at each point in x (shape: [num_samples,]).
    """
    n = data.shape[0]  # Number of data points
    d = x.shape[1]  # Dimensionality of the data

    # Convert bandwidth to tensor
    bandwidth_tensor = torch.tensor(bandwidth)

    # Calculate standardized distances for all data points
    u = (x[:, None, :] - data) / bandwidth_tensor

    # Compute Gaussian kernel contributions for all data points
    kernel_contributions = gaussian_kernel(u)

    # Sum contributions across all data points
    estimate = torch.mean(kernel_contributions, dim=1)

    # Normalize the density estimate by the number of points and the bandwidth raised to the dimensionality
    #estimate /= (bandwidth_tensor ** d)

    return estimate


In [ ]:
KDE(mals, bens, 1)

tensor([2.1148e-02, 5.8933e-04, 8.9441e-06, 9.6431e-25, 5.8933e-04, 7.3179e-06,
        4.7786e-09, 6.8697e-03, 5.3102e-13, 1.0105e-11, 2.1148e-02, 1.3240e-04,
        5.8933e-04, 4.0438e-03, 6.3707e-06])

In [ ]:
KDE(bens, bens, bandwidth=1)

tensor([0.0896, 0.2576, 1.2275, 0.3074, 0.1159, 0.0922, 0.0962, 0.0885, 0.0885,
        0.2817, 0.2509, 0.1174, 0.8223, 0.1772, 0.1416, 1.2275, 0.0965, 0.0963,
        0.1081, 1.2275, 0.3726, 0.5248, 0.1287, 0.1023, 0.1586, 0.1048, 0.0987,
        0.5699, 0.1095, 0.5281, 0.2011, 0.0985, 0.1772, 0.0922, 0.2133, 0.1495,
        0.1220, 0.0885, 0.1291, 0.7443, 0.2154, 0.7292, 0.9554, 0.1752, 0.3154,
        0.1890, 0.0885, 0.1020, 0.0933, 0.0948, 0.0887, 0.1746, 0.4055, 0.0899,
        0.1966, 0.5768, 0.3806, 0.0915, 0.1892, 0.0894, 1.0894, 0.5281, 0.1526,
        0.0885, 0.0888, 0.0886, 0.0885, 0.5711, 0.0989, 0.0885, 0.4786, 0.3597,
        0.1316, 0.7895, 0.0885, 1.2275, 0.0887, 0.0913, 0.1646, 0.4784, 0.1073,
        0.8008, 0.1017, 0.0902, 0.1754, 0.0977, 0.2193, 0.1349, 0.4773, 0.0885,
        0.2258, 0.7292, 0.0907, 1.0894, 0.0934, 0.1003, 0.0885, 0.0955, 0.1144,
        0.1290, 0.1110, 1.2275, 0.1131, 0.1185, 0.1022, 0.1726, 0.1280, 0.0885,
        0.8005, 0.0999, 0.0886, 0.0913, 

In [ ]:
def get_loss_kde(adv_x,y,model,benigns, bandwidth, penalty_factor):
    criterion = nn.CrossEntropyLoss(reduction='none')
    outputs = model(adv_x)
    n = mals.size(0)
    ce = criterion(outputs, y.view(-1).long())
    print('ce: ', ce)
    kde = KDE(adv_x, benigns, bandwidth)
    print('kde : ', kde)
    loss_no_reduction = ce + penalty_factor * kde
    _, predicted = torch.topk(outputs, k=1)
    done = (predicted != y).squeeze()

    return loss_no_reduction, done

In [ ]:
torch.zeros(15,dtype=torch.long)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
mals.size(0)

15

In [ ]:
for x,y in test_loader:
  print(x.shape)
  break

bens = x[y.squeeze()==0].to(torch.float32)
mals = x[y.squeeze()==1].to(torch.float32)
bens_label = y[y.squeeze()==0]
mals_label = y[y.squeeze()==1]
print(bens.shape)
print(mals.shape)

torch.Size([128, 10000])
torch.Size([113, 10000])
torch.Size([15, 10000])


In [ ]:
KDE(mals, bens, 0.5)

tensor([3.1386e-08, 9.6793e-15, 2.0732e-21, 0.0000e+00, 9.6793e-15, 3.7610e-19,
        5.5201e-33, 7.6521e-08, 0.0000e+00, 9.8133e-42, 3.1386e-08, 6.1342e-14,
        9.6793e-15, 3.9919e-11, 2.0517e-22])

In [ ]:
KDE(bens, bens, 0.5)

tensor([0.0885, 0.0888, 0.5273, 0.1056, 0.0885, 0.0885, 0.0885, 0.0885, 0.0885,
        0.1129, 0.1037, 0.0885, 0.1585, 0.1770, 0.0885, 0.5273, 0.0885, 0.0885,
        0.0885, 0.5273, 0.0895, 0.0940, 0.0885, 0.0885, 0.0888, 0.0885, 0.0885,
        0.0972, 0.0885, 0.0981, 0.0885, 0.0885, 0.1770, 0.0885, 0.0920, 0.0885,
        0.0885, 0.0885, 0.0885, 0.1246, 0.0886, 0.2006, 0.1867, 0.1021, 0.0888,
        0.0885, 0.0885, 0.0885, 0.0885, 0.0885, 0.0885, 0.0885, 0.0911, 0.0885,
        0.1124, 0.0984, 0.0895, 0.0885, 0.0902, 0.0885, 0.2771, 0.0981, 0.0901,
        0.0885, 0.0885, 0.0885, 0.0885, 0.0972, 0.0885, 0.0885, 0.0923, 0.0898,
        0.0885, 0.1291, 0.0885, 0.5273, 0.0885, 0.0885, 0.0885, 0.0935, 0.0885,
        0.1582, 0.0885, 0.0885, 0.1021, 0.0885, 0.0888, 0.0885, 0.0935, 0.0885,
        0.0906, 0.2006, 0.0885, 0.2771, 0.0885, 0.0885, 0.0885, 0.0885, 0.0885,
        0.0886, 0.0885, 0.5273, 0.0885, 0.0885, 0.0885, 0.0887, 0.0886, 0.0885,
        0.1582, 0.0885, 0.0885, 0.0885, 

In [ ]:

.to(torch.float32)

In [ ]:
loss,done = get_loss(mals,mals_label,model_AT_rFGSM,bens, 1., 100)

ce:  tensor([-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, 9.2741e-05, -0.0000e+00, -0.0000e+00, -0.0000e+00, 1.1325e+01,
        -0.0000e+00, -0.0000e+00, -0.0000e+00], grad_fn=<NllLossBackward0>)
kde :  tensor([2.1148e-02, 5.8933e-04, 8.9441e-06, 9.6431e-25, 5.8933e-04, 7.3179e-06,
        4.7786e-09, 6.8697e-03, 5.3102e-13, 1.0105e-11, 2.1148e-02, 1.3240e-04,
        5.8933e-04, 4.0438e-03, 6.3707e-06])


In [ ]:
loss.mean()

tensor(0.3874, grad_fn=<MeanBackward0>)

In [ ]:
k=100
norm='l1'
is_report_loss_diff = 'True'
bandwidth = 1.
penalty_factor = 100.
x = mals
y = mals_label
step_length = 1.0

model_AT_rFGSM.eval()

# Compute natural loss
loss_natural, _ = get_loss(x,y,model_AT_rFGSM,bens, bandwidth, penalty_factor)

# Initialize starting point
x_next = x.clone()
#x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

# Multi-step PGD
for t in range(k):
    # Forward pass
    x_var = x_next.clone().detach().requires_grad_(True)
    loss, _ = get_loss(x_var,y,model_AT_rFGSM,bens, bandwidth, penalty_factor)

    # Compute gradient
    grad_vars = torch.autograd.grad(loss.mean(), x_var)
    gradients = grad_vars[0].data

    #pos_insertion = (x_var <= 0.5) * 1 * insertion_array
    pos_insertion = (x_var <= 0.5) * 1
    grad4insertion = (gradients >= 0) * pos_insertion * gradients

    pos_removal = (x_var > 0.5) * 1 * removal_array
    grad4removal = (gradients < 0) * pos_removal * gradients

    gradients = grad4removal + grad4insertion
    #gradients =  grad4insertion

    # Norm
    if norm == 'linf':
        perturbation = torch.sign(gradients).float()

    elif norm == 'l2':
        l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
        perturbation = (gradients / l2norm).float()
        #perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm).float()
        #perturbation = torch.maximum(torch.tensor(-1., dtype=x.dtype, device=x.device), perturbation).float()
        #perturbation[torch.isnan(perturbation)] = 0.
        #perturbation[torch.isinf(perturbation)] = 1.

    elif norm == 'l1':
        # consider just features of a sample which are not updated yet(because our update is 0to1 or 1to0 not stepwise)
        un_mod = torch.abs(x_next - x_var) <= 1e-6
        gradients = gradients * un_mod

        val, _ = torch.topk(torch.abs(gradients), 1)
        perturbation = (torch.abs(gradients) >= val.expand_as(gradients)).float() * torch.sign(gradients).float()
        # stop perturbing the examples that are successful to evade the victim
        _, done = get_loss(x_next,y,model_AT_rFGSM,bens, bandwidth, penalty_factor)
        if torch.all(done):
            break
        print(done.shape)
        perturbation[done] = 0.

    else:
        raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

    # Update x_next
    x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

# Rounding step
if random:
    round_threshold = torch.rand(x_next.size())
x_next = round_x(x_next, round_threshold=round_threshold)

# Feasible projection
#x_next = or_float_tensors(x_next, x)

# Compute adversarial loss
loss, _ = get_loss(x_next,y,model_AT_rFGSM,bens, bandwidth, penalty_factor)
loss_adv = loss.data
if is_report_loss_diff:
    outputs = model_AT_rFGSM(x_next)
    _, predicted = torch.topk(outputs, k=1)
    done = (predicted != y).squeeze()
    print(f"PGD {norm}: Attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

# Replace with natural if adversarial loss is higher
replace_flag = (loss_adv < loss_natural).squeeze()
x_next[replace_flag] = x[replace_flag]


In [ ]:
def gkde(x, y, model,bens, bandwidth, penalty_factor, insertion_array, removal_array, k=25, step_length=0.02, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    Projected Gradient Descent (PGD) adversarial attack.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :param initial_rounding_threshold: Threshold parameter for rounding the initial x_next
    :param round_threshold: Threshold parameter for rounding
    :param random: Flag to generate random thresholds
    :param is_report_loss_diff: Flag to report loss difference
    :param is_sample: Flag to sample randomly from the feasible area
    :return: The adversarial version of x (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural, _ = get_loss_kde(x,y,model,bens, bandwidth, penalty_factor)

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        #y_model = model(x_var)
        loss,_ = get_loss_kde(x_var,y,model,bens, bandwidth, penalty_factor)

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        gradients = grad_vars[0].data

        #pos_insertion = (x_var <= 0.5) * 1 * insertion_array
        pos_insertion = (x_var <= 0.5) * 1
        grad4insertion = (gradients >= 0) * pos_insertion * gradients

        pos_removal = (x_var > 0.5) * 1 * removal_array
        grad4removal = (gradients < 0) * pos_removal * gradients

        gradients = grad4removal + grad4insertion
        #gradients =  grad4insertion

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients).float()

        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = (gradients / l2norm).float()
            #perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm).float()
            #perturbation = torch.maximum(torch.tensor(-1., dtype=x.dtype, device=x.device), perturbation).float()
            #perturbation[torch.isnan(perturbation)] = 0.
            #perturbation[torch.isinf(perturbation)] = 1.

        elif norm == 'l1':
            # consider just features of a sample which are not updated yet(because our update is 0to1 or 1to0 not stepwise)
            un_mod = torch.abs(x_next - x_var) <= 1e-6
            gradients = gradients * un_mod

            val, _ = torch.topk(torch.abs(gradients), 1)
            perturbation = (torch.abs(gradients) >= val.expand_as(gradients)).float() * torch.sign(gradients).float()
            # stop perturbing the examples that are successful to evade the victim
            _, done = get_loss_kde(x_next,y,model,bens, bandwidth, penalty_factor)
            if torch.all(done):
                break
            perturbation[done] = 0.

        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size())
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    #x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv, done = get_loss_kde(x_next,y,model,bens, bandwidth, penalty_factor)
    loss_adv = loss_adv.data

    if is_report_loss_diff:
        print(f"PGD {norm}: Attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next

In [ ]:
def get_loss_kde(adv_x,y,model,benigns, bandwidth, penalty_factor):
    criterion = nn.CrossEntropyLoss(reduction='none')
    outputs = model(adv_x)
    n = mals.size(0)
    ce = criterion(outputs, y.view(-1).long())
    print('ce: ', ce)
    kde = KDE(adv_x, benigns, bandwidth)
    print('kde : ', kde)
    loss_no_reduction = ce + penalty_factor * kde
    _, predicted = torch.topk(outputs, k=1)
    done = (predicted != y).squeeze()

    return loss_no_reduction, done

In [ ]:
kde :  tensor([9.4778e-02, 3.2176e-02, 2.3004e-03, 2.7972e-08, 3.2176e-02, 3.9920e-04,
        2.9941e-06, 1.1326e-02, 3.3817e-07, 2.2469e-04, 9.4778e-02, 1.3240e-04,
        3.2176e-02, 1.8123e-02, 1.2042e-03])

In [ ]:
kde :  tensor([9.4778e-02, 3.2158e-02, 9.2619e-05, 1.7899e-08, 3.2158e-02, 3.9920e-04,
        2.9941e-06, 1.1326e-02, 3.3817e-07, 2.2469e-04, 9.4778e-02, 1.3240e-04,
        3.2158e-02, 1.8123e-02, 1.2042e-03])

In [ ]:
kde :  tensor([9.4778e-02, 3.2158e-02, 6.8205e-05, 2.7972e-08, 3.2158e-02, 3.9920e-04,
        2.9941e-06, 1.1326e-02, 3.3817e-07, 6.2256e-07, 9.4778e-02, 1.3240e-04,
        3.2158e-02, 1.8123e-02, 1.2042e-03])

In [ ]:
kde :  tensor([9.4778e-02, 3.2158e-02, 6.8205e-05, 1.7899e-08, 3.2158e-02, 3.9920e-04,
        2.9941e-06, 1.1326e-02, 3.3817e-07, 6.2254e-07, 9.4778e-02, 1.3240e-04,
        3.2158e-02, 1.8123e-02, 2.0546e-04])

In [ ]:
gkde(mals,mals_label,model_AT_rFGSM,bens,1.,1000.,insertion_array, removal_array, k=200, step_length=0.05, norm='l2', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)

ce:  tensor([-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, 9.2741e-05, -0.0000e+00, -0.0000e+00, -0.0000e+00, 1.1325e+01,
        -0.0000e+00, -0.0000e+00, -0.0000e+00], grad_fn=<NllLossBackward0>)
kde :  tensor([2.1148e-02, 5.8933e-04, 8.9441e-06, 9.6431e-25, 5.8933e-04, 7.3179e-06,
        4.7786e-09, 6.8697e-03, 5.3102e-13, 1.0105e-11, 2.1148e-02, 1.3240e-04,
        5.8933e-04, 4.0438e-03, 6.3707e-06])
ce:  tensor([-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, 9.2741e-05, -0.0000e+00, -0.0000e+00, -0.0000e+00, 1.1325e+01,
        -0.0000e+00, -0.0000e+00, -0.0000e+00], grad_fn=<NllLossBackward0>)
kde :  tensor([2.1148e-02, 5.8933e-04, 8.9441e-06, 9.6431e-25, 5.8933e-04, 7.3179e-06,
        4.7786e-09, 6.8697e-03, 5.3102e-13, 1.0105e-11, 2.1148e-02, 1.3240e-04,
        5.8933e-04, 4.0438e-03, 6.3707e-06], grad_fn=<MeanBackward1>)
ce:  tensor([-0.0000e+00, -0.0000e+00, -0.0000e+0

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
pgd(mals, mals_label, model_AT_rFGSM, insertion_array, removal_array, k=200, step_length=0.05, norm='l2', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)

PGD l2: Attack effectiveness 86.667%.


tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])

In [ ]:
a = dfgsm_k(x.to(torch.float32), y, model_DNN, k=25)
b = dfgsm_k2(x.to(torch.float32), y, model_DNN, k=25)

dFGSM: attack effectiveness 100.000%.
dFGSM: attack effectiveness 100.000%.


In [ ]:
(a == b).sum()

tensor(10000)

In [ ]:
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
model_AT = MalwareDetectionModel().to(device)

attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':False}
adversarial_training(model_AT, train_loader, val_loader, adv_epochs=50, attack=dfgsm_k, **attack_param)

Epoch: 1/50 | Training loss (epoch level): 0.6947 | Train accuracy: 75.76%
	Val accuracy(without attack) 89.88% and accuracy(with attack) 12.65% under attack and overall accuracy 51.26%.
	Model select at epoch 1 with validation accuracy 51.26% and accuracy 12.65% under attack.
Epoch: 2/50 | Training loss (epoch level): 0.6497 | Train accuracy: 89.79%
	Val accuracy(without attack) 92.36% and accuracy(with attack) 15.88% under attack and overall accuracy 54.12%.
	Model select at epoch 2 with validation accuracy 54.12% and accuracy 15.88% under attack.
Epoch: 3/50 | Training loss (epoch level): 0.5433 | Train accuracy: 91.49%
	Val accuracy(without attack) 95.4% and accuracy(with attack) 42.11% under attack and overall accuracy 68.75%.
	Model select at epoch 3 with validation accuracy 68.75% and accuracy 42.11% under attack.
Epoch: 4/50 | Training loss (epoch level): 0.2668 | Train accuracy: 95.22%
	Val accuracy(without attack) 96.7% and accuracy(with attack) 72.76% under attack and overal

In [ ]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
model_evaluation(model_AT_rFGSM,test_loader,device)

Test Accuracy: 0.9893
Test balanced Accuracy: 0.9710
Precision: 0.9622
Recall: 0.9469
F1-score: 0.9545
True Positives (TP): 1070
True Negatives (TN): 8390
False Positives (FP): 42
False Negatives (FN): 60
False Negative Rate (FNR): 5.3097
False Positive Rate (FPR): 0.4981


In [ ]:
def PGD_Max2(x,y, model, insertion_array, removal_array, attack_list = ['linf', 'l2', 'l1'],steps_max=5, is_sample = False, varepsilon = 1e-20, is_report_loss_diff = False):
    """
    PGD_Max adversarial attack.

    Args:
        x: Input data tensor (shape: [samples, features])
        y: Ground truth labels tensor (shape: [samples])
        model: Neural network model
        attack_list: List of norms for attacks (default: ['linf', 'l2', 'l1'])
        steps_max: Maximum number of steps (default: 5)
        is_sample: Flag to sample randomly from the feasible area (default: False)
        vaρεpsilon: Tolerance for stopping condition (default: 1e-20)

    Returns:
        Adversarial version of input data (tensor)
    """
    batch_size = x.shape[0]
    norm_params = {
        'l1': {'k': 50, 'step_length': 1.0, 'is_report_loss_diff':is_report_loss_diff},
        'l2': {'k': 200, 'step_length': 0.05, 'is_report_loss_diff':is_report_loss_diff},
        'linf': {'k': 500, 'step_length': 0.002, 'is_report_loss_diff':is_report_loss_diff}
    }

    model.eval()
    with torch.no_grad():
        loss, done = get_loss(x,y,model) #shape:[samples],[samples]

    pre_loss = loss
    n = x.shape[0]
    adv_x = x.detach().clone().to(x.device)
    stop_flag = torch.zeros(n, dtype=torch.bool).to(x.device) #[samples]

    for t in range(steps_max):
      num_remaining  = (~stop_flag).sum().item()
      #print('number of remaining samples : ',num_remaining )
      if num_remaining  <= 0:
          break

      remaining_label = y[~stop_flag]
      pertbx = []

      for norm in attack_list:
          if norm in norm_params:
              params = norm_params[norm]
              perturbation = pgd(adv_x[~stop_flag], remaining_label, model, insertion_array, removal_array, norm=norm, is_sample=is_sample, **params)
              #print("the number of added features : ", (perturbation.sum() - adv_x[~stop_flag].sum())/len(adv_x[~stop_flag]))
              pertbx.append(perturbation)
          else:
              raise ValueError("Expected 'l1' or 'l2' or 'linf' norm.")


      # here pertbx.shape = a list of (number of attacks  ,(num_remaining ,features))
      pertbx = torch.vstack(pertbx)
      # here pertbx.shape = a tensor (num_remaining *number of attacks samples, features)

      with torch.no_grad():
        remaining_label_ext = torch.cat([remaining_label] * len(attack_list)) #(labels*number of attacks )
        loss, done = get_loss(pertbx, remaining_label_ext,model) #(labels*number of attacks )

        # for a sample, if there is at least one successful attack, we will select the one with maximum loss;
        # while if no attacks evade the victim successful, all perturbed examples are reminded for selection
        max_v = loss.amax()
        loss[done] += max_v

        loss = loss.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)
        done = done.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)

        success_flag = torch.any(done, dim=-1) #(num_remaining )

        pertbx = pertbx.reshape(len(attack_list), num_remaining , x.shape[1]).permute([1, 0, 2])#(num_remaining ,attacks,features)
        _, indices = loss.max(dim=-1) # ans:(samples), max loss among attacks which worked, and max loss among all attacks for sample , none of them worked
        adv_x[~stop_flag] = pertbx[torch.arange(num_remaining ), indices]
        a_loss = loss[torch.arange(num_remaining ), indices]
        pre_stop_flag = stop_flag.clone()
        stop_flag[~stop_flag] = (torch.abs(pre_loss[~stop_flag] - a_loss) < varepsilon) | success_flag
        pre_loss[~pre_stop_flag] = a_loss

    return adv_x

In [ ]:
# PGD-max(MaxMA)
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,'steps_max':1,'is_sample':False, 'varepsilon':1e-20, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, PGD_Max2, device, **attack_params)


Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.


In [ ]:
def pgd(x, y, model, insertion_array, removal_array, k=25, step_length=0.02, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    Projected Gradient Descent (PGD) adversarial attack.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :param initial_rounding_threshold: Threshold parameter for rounding the initial x_next
    :param round_threshold: Threshold parameter for rounding
    :param random: Flag to generate random thresholds
    :param is_report_loss_diff: Flag to report loss difference
    :param is_sample: Flag to sample randomly from the feasible area
    :return: The adversarial version of x (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        gradients = grad_vars[0].data

        #pos_insertion = (x_var <= 0.5) * 1 * insertion_array
        pos_insertion = (x_var <= 0.5) * 1
        grad4insertion = (gradients >= 0) * pos_insertion * gradients

        pos_removal = (x_var > 0.5) * 1 * removal_array
        grad4removal = (gradients < 0) * pos_removal * gradients

        gradients = grad4removal + grad4insertion
        #gradients =  grad4insertion

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients).float()

        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = (gradients / l2norm).float()
            #perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm).float()
            #perturbation = torch.maximum(torch.tensor(-1., dtype=x.dtype, device=x.device), perturbation).float()
            #perturbation[torch.isnan(perturbation)] = 0.
            #perturbation[torch.isinf(perturbation)] = 1.

        elif norm == 'l1':
            # consider just features of a sample which are not updated yet(because our update is 0to1 or 1to0 not stepwise)
            un_mod = torch.abs(x_next - x_var) <= 1e-6
            gradients = gradients * un_mod

            val, _ = torch.topk(torch.abs(gradients), 1)
            perturbation = (torch.abs(gradients) >= val.expand_as(gradients)).float() * torch.sign(gradients).float()
            # stop perturbing the examples that are successful to evade the victim
            outputs = model(x_next)
            _, predicted = torch.topk(outputs, k=1)
            done = (predicted != y).squeeze()
            if torch.all(done):
                break
            perturbation[done] = 0.

        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size())
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    #x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"PGD {norm}: Attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next

In [ ]:
# rFGSM
attack_params = {"k":50, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
#adv_predict(test_loader, model_DNN, dfgsm_k2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)

RuntimeError: Expected target size [0, 2], got [0]

In [ ]:
# PGD-l2
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,"k":50, "step_length":0.5, 'norm':'l2', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 37.17%.


In [ ]:
# PGD-linf
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,"k":500, "step_length":0.002, 'norm':'linf', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 42.04%.


In [ ]:
# PGD-l1
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,"k":500, "step_length":1., 'norm':'l1', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 36.55%.


In [ ]:
# Mimicry×10
attack_params = {"trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 25.31%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 94.16%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.04%.


In [ ]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 25.31%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.63%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 94.34%.


In [28]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

TypeError: mimicry() missing 1 required positional argument: 'removal_array'

In [ ]:
# Mimicry×100
attack_params = {"trials":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 24.6%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 92.39%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 92.3%.


In [ ]:
# Mimicry×1000
attack_params = {"trials":1000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 2.478%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 92.04%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 87.26%.


In [ ]:
# PGD-max(MaxMA)
attack_params = {'steps_max':1,'is_sample':False, 'varepsilon':1e-20, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, PGD_Max2, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 89.29%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 80.62%.


In [ ]:
# PGD-max(iMaxMA)
attack_params = {'steps_max':5,'is_sample':False, 'varepsilon':1e-20, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, PGD_Max2, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 87.96%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 74.16%.


In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.5, "linf": 0.0008}, "steps":700}
adv_predict(test_loader, model_AT_rFGSM, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 7.826%.
step-wise max: attack effectiveness 8.547%.
step-wise max: attack effectiveness 15.200%.
step-wise max: attack effectiveness 6.796%.
step-wise max: attack effectiveness 11.504%.
step-wise max: attack effectiveness 16.216%.
step-wise max: attack effectiveness 9.630%.
step-wise max: attack effectiveness 9.489%.
step-wise max: attack effectiveness 7.200%.
step-wise max: attack effectiveness 12.245%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 89.65%.


In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.2, "linf": 0.0005}, "steps":1000}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 9.565%.
step-wise max: attack effectiveness 7.692%.
step-wise max: attack effectiveness 16.800%.
step-wise max: attack effectiveness 10.680%.
step-wise max: attack effectiveness 12.389%.
step-wise max: attack effectiveness 20.721%.
step-wise max: attack effectiveness 11.852%.
step-wise max: attack effectiveness 10.949%.
step-wise max: attack effectiveness 8.800%.
step-wise max: attack effectiveness 12.245%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 87.88%.


In [ ]:
# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),

    (pgd, {'k': 100, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'k': 100, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (StepwiseMax_onestep2, {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

    # Add more attacks as needed
]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    print()  # Print an empty line for separation


Running attack: StepwiseMax_onestep2 with parameters: {'step_lengths': {'l1': 1.0, 'l2': 0.05, 'linf': 0.0008}, 'steps': 650}


TypeError: StepwiseMax_onestep2() missing 2 required positional arguments: 'insertion_array' and 'removal_array'